# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from citipy import citipy

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Aasiaat,68.7098,-52.8699,38.07,89,100,8.88,GL,1691688934
1,1,Kodinsk,58.6881,99.1844,58.23,83,78,2.71,RU,1691688934
2,2,Grytviken,-54.2811,-36.5092,26.28,96,95,7.65,GS,1691688935
3,3,Taoudenni,22.6783,-3.9836,114.10,8,8,13.00,ML,1691688935
4,4,Abashiri,44.0213,144.2697,67.35,95,100,4.27,JP,1691688935


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Distance",
    scale = 0.5,
    color = "Humidity",

) 


# Display the map plot
map_plot_1


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city_df = pd.DataFrame (city_data_df, columns = ["City", "Cloudiness", "Humidity", "Wind Speed", "Max Temp"])
ideal_city_df = ideal_city_df.loc[(ideal_city_df["Cloudiness"] == 0) & (ideal_city_df["Humidity"] < 50) & (ideal_city_df["Wind Speed"] < 5)]

# Drop any rows with null values
clean_ideal_city_df = ideal_city_df.dropna()

# Display sample data
clean_ideal_city_df

,City,Cloudiness,Humidity,Wind Speed,Max Temp
24,Cologne,0,49,2.30,76.87
34,Saint-Pathus,0,48,3.44,83.80
112,Bo`ka,0,25,4.32,81.10
126,Tamanghasset,0,10,3.44,94.91
151,Newman,0,21,0.43,83.23
249,Tamulte de las Sabanas,0,46,4.32,95.09
386,Pinheiral,0,36,3.40,87.15
400,Kizil-Arvat,0,18,3.40,93.11
500,Turpan,0,17,2.55,97.77
523,South Yuba City,0,48,4.61,90.99


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = pd.DataFrame (city_data_df, columns = ["City", "Country", "Lat", "Lng", "Humidity"])

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,Aasiaat,GL,68.7098,-52.8699,89,
1,Kodinsk,RU,58.6881,99.1844,83,
2,Grytviken,GS,-54.2811,-36.5092,96,
3,Taoudenni,ML,22.6783,-3.9836,8,
4,Abashiri,JP,44.0213,144.2697,95,
...,...,...,...,...,...,...
575,Eduardo Castex,AR,-35.9150,-64.2945,37,
576,Norderney,DE,53.7000,7.1500,77,
577,Aguilar de Campoo,ES,42.7945,-4.2589,55,
578,Huicheng,CN,23.0385,116.2899,87,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set base URL
base_url = "https://api.geoapify.com/v2/places"

# Set parameters to search for a hotel
radius = 10000

params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":limit
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
   
    # get latitude, longitude from the DataFrame
    
    Lat = row["Lat"]
    Lng = row["Lng"]
    
    params["filter"] = f"circle:{Lng},{Lat}, {radius}"
    params["bias"] = f"proximity:{Lng},{Lat}" 
    
   
    # Make and API request using the params dictionaty
    response = requests.get(base_url, params=params)
    name_address = response.json()    

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


Starting hotel search
Aasiaat - nearest hotel: No hotel found
Kodinsk - nearest hotel: No hotel found
Grytviken - nearest hotel: No hotel found
Taoudenni - nearest hotel: No hotel found
Abashiri - nearest hotel: No hotel found
Isafjordur - nearest hotel: No hotel found
As Sulayyil - nearest hotel: No hotel found
Tralee - nearest hotel: No hotel found
Margaret River - nearest hotel: No hotel found
Kapuskasing - nearest hotel: No hotel found
Amursk - nearest hotel: No hotel found
Jinghong - nearest hotel: No hotel found
Port Saint John's - nearest hotel: No hotel found
Stanley - nearest hotel: No hotel found
Xinyuan - nearest hotel: No hotel found
Barra de Santo Antônio - nearest hotel: No hotel found
Saint Paul Harbor - nearest hotel: No hotel found
Hawaiian Paradise Park - nearest hotel: No hotel found


KeyboardInterrupt: 

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Distance",
    scale = 0.5,
    color = "Hotel Name",

) 


# Display the map plot
map_plot_2